In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.mongodb.spark:mongo-spark-connector_2.11:2.2.5 pyspark-shell"

import findspark
findspark.add_packages(["org.mongodb.spark:mongo-spark-connector_2.11:2.2.5"])
findspark.init()

In [2]:
import sys
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram
from pyspark.sql.types import IntegerType, MapType, StringType, BooleanType

from pyspark.sql.functions import (
    udf,
    split,
    explode,
    col,
    lower,
    count,
    collect_list,
    concat,
    create_map,
    lit,
    countDistinct,
)

In [3]:
mongo_user = "dev"
mongo_password = "dev12345"
mongo_url = "ds123454.mlab.com:23454/macri-discursos"
monto_url_stats = "ds115283.mlab.com:15283/discursos-stats"

uri_in = "mongodb://{}:{}@{}.speeches".format(mongo_user, mongo_password, mongo_url)

uri_out = "mongodb://{}:{}@{}.wordcount_out".format(
    mongo_user, mongo_password, mongo_url
)

In [4]:
spark = (
    SparkSession.builder.appName("discursos.counter")
    .config("spark.mongodb.input.uri", uri_in)
    .config("spark.mongodb.output.uri", uri_out)
    .getOrCreate()
)

In [5]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [8]:
df.take(1)

[Row(__v=0, _id=Row(oid='5c032582c104041f38a6f985'), content='BIENVENIDA DEL PRESIDENTE DE LA NACIÓN, MAURICIO MACRI, AL PRESIDENTE DE LOS ESTADOS UNIDOS DE AMÉRICA, DONALD TRUMP, EN CASA ROSADA\n\nMACRI.- Quiero darle la bienvenida al presidente Donald Trump a nuestro país, en su primera visita a América Latina y primera obviamente a la Argentina.\n\nRecién veníamos subiendo la escalera y le dije “tardamos 30 años, Donald, en lograr convencerte de que vengas”, porque hace más de 30 años que lo invito a visitar mi Ciudad y nuestro lindo país. Tuvimos que esperar que llegue a Presidente y yo a Presidente para que se logre, pero realmente estamos muy contentos los argentinos de tenerte acá en nuestro país.\n\nY es un gran momento para agradecer, agradecer el enorme apoyo que hemos recibido por parte de los Estados Unidos, de tu Gobierno, especialmente en este último año donde estamos atravesando momentos difíciles, y con el apoyo de ustedes y el acuerdo que se facilitó con el Fondo, hemo